In [50]:
import requests
import pandas as pd

BINANCE_URL = "https://api.binance.com/api/v3/klines"


def fetch_klines(symbol: str, interval: str, limit: int):
    """
    Descarga las últimas `limit` velas para un símbolo e intervalo.
    interval: '1d' (diario) o '1w' (semanal)
    """
    params = {
        "symbol": symbol,
        "interval": interval,
        "limit": limit,
    }
    resp = requests.get(BINANCE_URL, params=params, timeout=10)
    resp.raise_for_status()
    data = resp.json()
    if not data:
        raise ValueError(f"Sin datos para {symbol} {interval}")

    df = pd.DataFrame(data, columns=[
        "open_time","open","high","low","close","volume",
        "close_time","qav","num_trades",
        "taker_buy_base","taker_buy_quote","ignore"
    ])
    df["date"] = pd.to_datetime(df["open_time"], unit="ms")
    df["close"] = df["close"].astype(float)
    df["volume"] = df["volume"].astype(float)
    df["taker_buy_base"] = df["taker_buy_base"].astype(float)
    df["taker_sell_base"] = df["volume"] - df["taker_buy_base"]
    df = df.sort_values("date").reset_index(drop=True)
    return df


def compute_opportunity_signal(
        df: pd.DataFrame,
        lookback: int = 7,
        sell_pressure_threshold: float = 1.3,
        drop_threshold_pct: float = 3.0,
        volume_mult_threshold: float = 1.3
):
    """
    Usa la ÚLTIMA vela de df como "hoy" y las anteriores como contexto.

    Señal = volumen alto *y* (caída de precio o presión vendedora alta).

    Devuelve (signal: bool, metrics: dict)
    """
    if len(df) < lookback + 2:
        return False, {"reason": "pocos datos"}

    # Última vela = la actual
    last = df.iloc[-1]
    prev = df.iloc[-(lookback+1):-1]  # lookback velas previas

    prev_avg_close = prev["close"].mean()
    prev_avg_vol = prev["volume"].mean()

    last_close = last["close"]
    last_vol = last["volume"]

    taker_buy = last["taker_buy_base"]
    taker_sell = last["taker_sell_base"]

    sell_pressure = (taker_sell + 1e-9) / (taker_buy + 1e-9)
    drop_pct = (prev_avg_close - last_close) / prev_avg_close * 100 if prev_avg_close > 0 else 0
    vol_mult = last_vol / prev_avg_vol if prev_avg_vol > 0 else 1

    cond_drop = drop_pct >= drop_threshold_pct
    cond_sell = sell_pressure >= sell_pressure_threshold
    cond_vol = vol_mult >= volume_mult_threshold

    signal = cond_vol and (cond_drop or cond_sell)

    metrics = {
        "last_date": last["date"],
        "last_close": last_close,
        "prev_avg_close": prev_avg_close,
        "drop_pct": drop_pct,
        "sell_pressure": sell_pressure,
        "vol_mult": vol_mult,
        "cond_drop": cond_drop,
        "cond_sell": cond_sell,
        "cond_vol": cond_vol,
    }
    return signal, metrics


def check_opportunity_both_modes(
        symbols=None,
        lookback_daily: int = 7,
        lookback_weekly: int = 7,
        sell_pressure_threshold: float = 1.3,
        drop_threshold_pct: float = 3.0,
        volume_mult_threshold: float = 1.3,
        combine_mode: str = "or",  # "or" o "and"
):
    """
    Chequea oportunidad para cada símbolo usando SIEMPRE:
      - velas DIARIAS
      - velas SEMANALES

    combine_mode:
      - "or": señal combinada = daily OR weekly
      - "and": señal combinada = daily AND weekly
    """
    if symbols is None:
        symbols = ["BTCUSDT", "ETHUSDT"]

    print(f"Detector de oportunidad (daily + weekly) - modo combinado = {combine_mode}\n")

    results = {}

    for symbol in symbols:
        print(f"================ {symbol} ================")

        # ---- Diario ----
        df_d = fetch_klines(symbol, "1d", limit=lookback_daily + 10)
        sig_d, m_d = compute_opportunity_signal(
            df_d,
            lookback=lookback_daily,
            sell_pressure_threshold=sell_pressure_threshold,
            drop_threshold_pct=drop_threshold_pct,
            volume_mult_threshold=volume_mult_threshold,
        )

        print(">> MODO DIARIO (1d)")
        print(f"Última fecha:          {m_d['last_date']}")
        print(f"Precio cierre último:  {m_d['last_close']:.2f}")
        print(f"Promedio cierre prev.: {m_d['prev_avg_close']:.2f}")
        print(f"Caída vs promedio:     {m_d['drop_pct']:.2f}%")
        print(f"Presión vendedora:     {m_d['sell_pressure']:.2f}")
        print(f"Volumen relativo:      {m_d['vol_mult']:.2f}x")
        print(f"Señal diaria:          {sig_d}\n")

        # ---- Semanal ----
        df_w = fetch_klines(symbol, "1w", limit=lookback_weekly + 10)
        sig_w, m_w = compute_opportunity_signal(
            df_w,
            lookback=lookback_weekly,
            sell_pressure_threshold=sell_pressure_threshold,
            drop_threshold_pct=drop_threshold_pct,
            volume_mult_threshold=volume_mult_threshold,
        )

        print(">> MODO SEMANAL (1w)")
        print(f"Última semana:         {m_w['last_date']}")
        print(f"Precio cierre último:  {m_w['last_close']:.2f}")
        print(f"Promedio cierre prev.: {m_w['prev_avg_close']:.2f}")
        print(f"Caída vs promedio:     {m_w['drop_pct']:.2f}%")
        print(f"Presión vendedora:     {m_w['sell_pressure']:.2f}")
        print(f"Volumen relativo:      {m_w['vol_mult']:.2f}x")
        print(f"Señal semanal:         {sig_w}\n")

        # ---- Combinación ----
        if combine_mode == "and":
            combined = sig_d and sig_w
        else:  # "or" por defecto
            combined = sig_d or sig_w

        print(f"==> SEÑAL COMBINADA ({combine_mode.upper()}): {combined}\n")

        results[symbol] = {
            "daily_signal": sig_d,
            "daily_metrics": m_d,
            "weekly_signal": sig_w,
            "weekly_metrics": m_w,
            "combined_signal": combined,
        }

    return results


if __name__ == "__main__":
    # Ejemplo de uso rápido:
    _ = check_opportunity_both_modes(
        symbols=["BTCUSDT", "ETHUSDT"],
        lookback_daily=7,
        lookback_weekly=7,
        sell_pressure_threshold=1.3,
        drop_threshold_pct=3.0,
        volume_mult_threshold=1.3,
        combine_mode="or"   # probá también "and" si querés ser más exigente
    )


Detector de oportunidad (daily + weekly) - modo combinado = or

================ BTCUSDT ================
>> MODO DIARIO (1d)
Última fecha:          2025-12-06 00:00:00
Precio cierre último:  89446.25
Promedio cierre prev.: 90509.20
Caída vs promedio:     1.17%
Presión vendedora:     1.02
Volumen relativo:      0.36x
Señal diaria:          False

>> MODO SEMANAL (1w)
Última semana:         2025-12-01 00:00:00
Precio cierre último:  89446.24
Promedio cierre prev.: 101416.75
Caída vs promedio:     11.80%
Presión vendedora:     1.16
Volumen relativo:      0.79x
Señal semanal:         False

==> SEÑAL COMBINADA (OR): False

================ ETHUSDT ================
>> MODO DIARIO (1d)
Última fecha:          2025-12-06 00:00:00
Precio cierre último:  3037.22
Promedio cierre prev.: 3016.99
Caída vs promedio:     -0.67%
Presión vendedora:     1.11
Volumen relativo:      0.30x
Señal diaria:          False

>> MODO SEMANAL (1w)
Última semana:         2025-12-01 00:00:00
Precio cierre último:  3